Day 2: 간단한 단일 & 여러 테이블 SQL 명령
======================

In [1]:
%load_ext sql
%sql sqlite://

'Connected: @None'

새로운 테이블을 생성해봅시다. 그리고 데이터를 넣어보고 질의문을 써보자. 

In [2]:
%%sql drop table if exists product;
create table product(
       pname        varchar primary key, -- name of the product
       price        money,               -- price of the product
       category     varchar,             -- category
       manufacturer varchar NOT NULL     -- manufacturer
);
insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

위의 명령의 실행 결과를 살펴보자. 

In [3]:
%sql select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
MultiTouch,203.99,Household,Hitachi


SQL의 몇 가지 *용어*.
--------------------------
* 이 테이블의 _이름_: product.
* 테이블의 각 줄: _row_ 또는 _tuple_. 
* 모든 터플은 fields 또는 _속성(attributes)_ 을 갖음.
* 테이블의 줄 수는 _cardinality_ 라고 부름. 속성의 개수는 _arity_ 라고 함

스키마의 관용 용법
-----------------
* product의 스키마는 다음과 같다:

> product(__pname__, price, category, manufacturer)

일반적으로 밑 줄 쳐있는 속성은 _키(key)_ 다. 여기서는 볼드로 표현됨

* 이 경우 pname 가 키가 된다. 만약 회사마다 제품명이 겹치기 때문에 회사명을 포함해야 유일한 경우 다음과 같이 작성할 수 있다.
> product(__pname__, price, category, __manufacturer__)


테이블의 설명
----------------
* A tuple = a record
  * Restriction: 모든 속성 값은 원자성을 갖어야 한다.
  * SQL은 다양한 원자 데이터 형을 갖고 있다. 참고 [here](http://www.postgresql.org/docs/9.4/static/datatype.html)


* A table = 터플의 (다중) 집합
  * 다중 집합은 리스트와 같다 …
  * ... 리스트와 달리 순서가 없다: 
    *  first(),  next(),  last() 같은 명령이 적용 되지 않는다.

# 어디까지 왔나?
* database 생성하기 -- 완료!

* 간단한 질의문 -- 지금부터!

* 하나의 테이블 이상을 활요하는 질의문 -- 다음에!


# 간단한 질의문을 작성해보자! 
* SQL의 간단한 용법을 예와 함께 다뤄보자
* 웹에 좋은 SQL 튜토리얼이 많이 있다. 여기서 다루는 내용은 맛보기일 뿐이다.

> SELECT (속성들)<br>
> FROM (하나 또는 그 이상의 테이블)<br>
> WHERE (조건문)

간단한 SELECT-FROM-WHERE (SFW) 블럭이다. 예를 보자.!

In [4]:
%%sql
SELECT * from Product 
WHERE category='Gadgets' and price > 20.0;

 * sqlite://
Done.


pname,price,category,manufacturer
PowerGizmo,29.99,Gadgets,GizmoWorks


질의문의 결과 중 어떤 속성의 값들만 출력하는 *projection*의 예를 살펴보자. 

In [5]:
%%sql 
SELECT Pname, Price, Manufacturer
FROM Product;

 * sqlite://
Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks
SingleTouch,149.99,Canon
MultiTouch,203.99,Hitachi


* 실행의 결과 *역시* 테이블이고, 그 스키마는 
> Answer(pname, price, manufacturer)

* 당연히 선택문과 projection을 같이 사용할 수 있다.

In [6]:
%%sql
SELECT Pname, Price, Manufacturer
FROM Product
WHERE category='Gadgets';

 * sqlite://
Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks


테이블에 대한 질의문의 결과 역시 테이블이다
----------------------------------------------
* 질의 언어는 *compositional* 성질을 갖고 있기 때문
* 실제로 질의문의 결과는 테이블이다.
* 다음의 이상한 질의문이 무엇을 찾는지 알겠는가?

In [7]:
%%sql
SELECT * FROM Product;

SELECT
    p.manufacturer, p.pname, p.price
FROM 
    (SELECT distinct p0.Manufacturer
     FROM Product p0
     WHERE p0.price < 20.00) cp, -- 중첩 질의를 사용하고 있음!
    Product p
WHERE 
    p.manufacturer = cp.manufacturer and p.price > 20.00

 * sqlite://
Done.
Done.


manufacturer,pname,price
GizmoWorks,PowerGizmo,29.99


SQL의 세부 내용
--------------

* 어떤 요소들은 대소문자를 구분한다 (프로그램이라고 생각해보자):
  * 같음: SELECT  Select  select
  * 같음: Product   product
  * 다름: ‘Seattle’  ‘seattle’
  

* 상수 (홑 따옴표)
  * ‘abc’  - 옮음
  * “abc” - 그름


LIKE 연산자
=========

LIKE 연산자는 문자열 검색에 사용된다. 와일드카드도 사용가능하다. 용법은 다음과 같다.
    
> SELECT *
> FROM Products
> WHERE pname like '%gizmo%'

* % 은 몇 개의 문자든 상관없음을 뜻함
* \_ 한 글자 검사
* like 연산자는 대소문자 구분함

In [8]:
%%sql
SELECT *  FROM product
where pname LIKE '%Gizmo%'

 * sqlite://
Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks


중복 제거
---------------------
* 중복을 기대하지 않았는데, 중복된 결과 값을 얻으면 당황하게 된다.
  * _다중 집합_ 이라고 했던 것을 기억하자!

In [9]:
%sql SELECT category from product;

 * sqlite://
Done.


category
Gadgets
Gadgets
Photography
Household


In [10]:
%%sql 
-- 중복을 제거하는 것은 간단하다.  distinct 키워드를 쓰면 된다.
SELECT DISTINCT category from product;

 * sqlite://
Done.


category
Gadgets
Photography
Household


결과의 순서 정렬하기
---------------------
* 결과 값이 정렬되어 있기를 바라는 경우 다음의 예를 보자.


In [11]:
%%sql
-- 결과의 정렬을 원할 때
-- order by 라는 명령을 사용하고, 오름차순 기본이다.
SELECT   pname, price, manufacturer
FROM     Product
WHERE    price > 50
ORDER BY  price, pname

 * sqlite://
Done.


pname,price,manufacturer
SingleTouch,149.99,Canon
MultiTouch,203.99,Hitachi


In [12]:
%%sql
-- 결과의 정렬을 원할 때
-- 아래와 같이 정렬할 수도 있다. 각 값을 개별적으로 구분하기
SELECT   price, manufacturer
FROM     Product
-- 정렬의 순서가 "사전식"으로 표현되어 있다. ASC는 오름차순, DESC는 내림차순
ORDER BY   manufacturer ASC, price DESC

 * sqlite://
Done.


price,manufacturer
149.99,Canon
29.99,GizmoWorks
19.99,GizmoWorks
203.99,Hitachi


Joins!
------
이 번에는 두 개의 테이블을 활용하는 좀 더 복잡한 질의문을 다뤄보자. 

* 회사명, 주가, 본사가 있는 나라로 정리된 테이블을 살펴보자
> company(__cname__, stockprice, country)
 
* products 테이블은 이 후에 일관성을 다룰 때 다시 보기로 한다.

In [13]:
%%sql
drop table if exists product; -- 존재하면 삭제한다. 왜 그래야 하는지 아래에서 확인해보자.
drop table if exists company;
create table company (
    cname varchar primary key, -- 회사 명으로 유일하게 회사들을 구분할 수 있음
    stockprice money, -- 주가 정보는 money에 등록 
    country varchar); -- country는 문자열
insert into company values ('GizmoWorks', 25.0, 'USA');
insert into company values ('Canon', 65.0, 'Japan');
insert into company values ('Hitachi', 15.0, 'Japan');

 * sqlite://
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [14]:
%sql select * from company;

 * sqlite://
Done.


cname,stockprice,country
GizmoWorks,25,USA
Canon,65,Japan
Hitachi,15,Japan


외부 키 (Foreign Key) 제약 조건
-----------------------
* products 테이블을 아래처럼 만든다고 해보자.

> Product(pname, price, category, manufacturer)

* 이상한 점: 어떤 업체가 제품을 팔고 있지만, 우리의 회사 테이블에 나타나지 않는다.
* 이런 문제를 없애기 위해 _foreign keys_ 제약 조건을 소개한다. 

products 의 회사 이름은 company 테이블의 cname을 가리키고 있다. SQL로 표현해보자. 핵심 문장은 다음과 같다.
> foreign key (manufacturer) references company(cname)

  * cname은 company 테이블에 있어야 한다.
  * Keys 와 Foreign keys 는 _늘_ 사용된다. 
    * PKs 과 FKs 라고 일반적으로 표기된다.

In [15]:
%%sql drop table if exists product;
pragma foreign_keys = ON; -- WARNING sqlite에서는 기본이 off
create table product(
       pname varchar primary key, --  product의 이름
       price money, -- 제품의 가격
       category varchar, -- category
       manufacturer varchar, -- manufacturer
       foreign key (manufacturer) references company(cname));

insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

 * sqlite://
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

foreign keys 는 _제약 조건_ 이다.
> 테이블에 없는 회사 이름을 쓴다면 어떻게 될까?


In [19]:
try:
    %sql insert into product values('MultiTouch', 203.99, 'Household', 'Google');
except Exception as e:
    print (e)
    print ("Rejected!")

 * sqlite://
(sqlite3.IntegrityError) UNIQUE constraint failed: product.pname [SQL: "insert into product values('MultiTouch', 203.99, 'Household', 'Google');"] (Background on this error at: http://sqlalche.me/e/gkpj)
Rejected!


In [17]:
%%sql
-- 갱신 질의가 거절됨!
select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
PowerGizmo,29.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
MultiTouch,203.99,Household,Hitachi


Foreign Keys 와 삭제
=============

* 회사를 삭제할 때 어떤 일이 생길까? 옵션은 세 개가 있다:
  * 삭제 불허 (기본)
  * 모든 제품 삭제 ("`on delete cascade`" 명령 추가 시)
  * NULL 표기로 인한 예외
  

**옵션 하나(기본)- 삭제 불허**

In [22]:
try:
    %sql delete from company where cname = 'Hitachi';
except Exception as e:
    print (e)
    print ("Disallowed!")

 * sqlite://
(sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: "delete from company where cname = 'Hitachi';"] (Background on this error at: http://sqlalche.me/e/gkpj)
Disallowed!


**옵션 둘: 회사 삭제 시 회사와 연관된 모든 제품 삭제**

product 테이블을 생성할 때 foreign key 제약 조건 절에 다음을 추가해보자.

> foreign key (manufacturer) references company(cname) on delete cascade

이제, 어떤 회사가 삭제되면, 해당 회사에 외부 키로 연결된 모든 제품들이 모두 삭제 된다. 


Joins!
------
> Product (<u>pname</u>,  price, category, manufacturer)<br>
> Company (<u>cname</u>, stockPrice, country)

우리가 답을 얻기 원기 원하는 질문은 다음과 같다:

> 일본에서 제조된 $200 이하의 모든 제품을 찾아보자.
> 그리고 그 이름과 가격을 출력하자

product에는 위치가 없고 manufacturers에는 가격이 없다. 각 테이블의 어떤 정보가 필요하다.

In [23]:
%%sql
SELECT pname, price
FROM product, company
where manufacturer=cname and country='Japan' and price <= 200;

 * sqlite://
Done.


pname,price
SingleTouch,149.99


모듈러 방식으로 join을 사용하는 방법을 알아보자.


In [24]:
%%sql -- Part 1: 일본 회사의 집합
SELECT distinct cname -- distinct가 필요한가?
from company where country='Japan';

 * sqlite://
Done.


cname
Canon
Hitachi


In [25]:
%%sql -- Part 2: $200 이하의 제품들
select distinct pname, price, manufacturer
from product
where price <= 200;

 * sqlite://
Done.


pname,price,manufacturer
Gizmo,19.99,GizmoWorks
PowerGizmo,29.99,GizmoWorks
SingleTouch,149.99,Canon


In [26]:
%%sql -- 중첩 SFW 질의문으로 결합해보자. 벡터곱인가?
SELECT * 
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 200) CheapProducts,
  (SELECT DISTINCT cname
   FROM company
   WHERE country='Japan') JapaneseProducts;

 * sqlite://
Done.


pname,price,manufacturer,cname
Gizmo,19.99,GizmoWorks,Canon
Gizmo,19.99,GizmoWorks,Hitachi
PowerGizmo,29.99,GizmoWorks,Canon
PowerGizmo,29.99,GizmoWorks,Hitachi
SingleTouch,149.99,Canon,Canon
SingleTouch,149.99,Canon,Hitachi


In [27]:
%%sql
-- join으로 결합해보자!
SELECT DISTINCT pname, price
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 200) CheapProducts,
  (SELECT distinct cname
   FROM company
   WHERE country='Japan') JapaneseProducts
WHERE cname = manufacturer;

 * sqlite://
Done.


pname,price
SingleTouch,149.99


배울점
--------
* 어떤 질의의 답을 얻기 위해 잠재적으로 _여러 논리적 동치 질의_ 가 있을 수 있다.
    * 질의 최적화에 대해 다루게 될 것이다.
    * 시험에 질의문 작성 문제가 나온다면 가장 간단한 방법으로 작성하자. 위처럼 단계적으로 처리하면 좋음

join으로 인한 중복 결과 값
--------------------------

join의 결과로 중복된 값들이 나올 수 있다는 것을 기억해야 한다.

In [28]:
%%sql -- 중복된 결과
SELECT Country
FROM Product, Company
WHERE  Manufacturer=CName AND Category='Gadgets';

 * sqlite://
Done.


country
USA
USA
